Ici, la méthodo sera différente : 5 parquet vont être produit pour 2020. 

In [1]:
from lxml import etree
import pandas as pd 
import os
import sqlite3
import glob 

In [5]:
def parsing_fichier_etree(chemin) : 
 ''' parse en utilisant lxml plutôt que xmltodict '''
 with open(chemin, "r") as fichier_ouvert:
  arbre = etree.parse(fichier_ouvert)
  racine = arbre.getroot()
  enfants = racine.getchildren()
 return enfants

def extraction_metadonnees(chemin) : 
 ''' Extrait l'année et la nomenclature complète depuis l'adresse html'''
 annee = chemin.split('/')[2]
 nomenclature = chemin.split('/')[3].split('.')[0]

 return annee, nomenclature

In [9]:
def extraction_nature(enfants, annee, nomenclature) -> pd.DataFrame: 
 ''' Permet de récupérer les lignes de la branche Nature ( Nature et ContNat )'''
 nature_chapitre = enfants[0].getchildren()[0].xpath(".//*[@Code]")
 nature_compte = enfants[0].getchildren()[1].xpath(".//*[@Code]")
 liste_nature_chapitre = []
 liste_nature_compte = []

 for i in nature_chapitre :
  liste_nature_chapitre.append(i.attrib)
 df_nature_chapitre = pd.DataFrame(liste_nature_chapitre)
 df_nature_chapitre['Exer'] = annee
 df_nature_chapitre['Nomenclature'] = nomenclature

 for i in nature_compte : 
  liste_nature_compte.append(i.attrib)
 df_nature_compte = pd.DataFrame(liste_nature_compte)
 df_nature_compte['Exer'] = annee
 df_nature_compte['Nomenclature'] = nomenclature

 return df_nature_chapitre, df_nature_compte

def extraction_fonction(enfants, annee, nomenclature) -> pd.DataFrame:
 ''' Permet de récupérer les lignes de la branche Fonction ( Fonction, Fonction Compte et Fonction ref, ret, machin )'''
 fonction_chapitre = enfants[1].getchildren()[0].xpath(".//*[@Code]")
 fonction_compte = enfants[1].getchildren()[1].xpath(".//*[@Code]")
 fonction_ret = enfants[1].getchildren()[2].xpath(".//*[@Code]")

 liste_fonction_chapitre = []
 liste_fonction_compte = []
 liste_fonction_ret = []

 for i in fonction_chapitre:
    liste_fonction_chapitre.append(i.attrib)
 df_fonction_chapitre = pd.DataFrame(liste_fonction_chapitre)
 df_fonction_chapitre['Exer'] = annee
 df_fonction_chapitre['Nomenclature'] = nomenclature

 for i in fonction_compte:
    liste_fonction_compte.append(i.attrib)
 df_fonction_compte = pd.DataFrame(liste_fonction_compte)
 df_fonction_compte['Exer'] = annee
 df_fonction_compte['Nomenclature'] = nomenclature

 for i in fonction_ret:
    liste_fonction_ret.append(i.attrib)
 df_fonction_ret = pd.DataFrame(liste_fonction_ret)
 df_fonction_ret['Exer'] = annee
 df_fonction_ret['Nomenclature'] = nomenclature

 return df_fonction_chapitre, df_fonction_compte, df_fonction_ret



In [12]:
def xml_to_parquet_plan_de_compte(annee) : 
 lnature_chapitre = []
 lnature_compte = []

 l_fonction = []
 l_contfon = []
 l_fonction_ref = []

 for plan_de_compte in glob.glob(os.path.join(f'./stockage_plan_de_compte/{annee}', "*.xml")) :
  enfant = parsing_fichier_etree(plan_de_compte)
  annee, nomenclature = extraction_metadonnees(plan_de_compte)

  df_nature_chapitre, df_nature_compte = extraction_nature(enfant, annee, nomenclature)
  lnature_chapitre.append(df_nature_chapitre)
  lnature_compte.append(df_nature_compte)

  df_fonction, df_contfon, df_fonction_ref =  extraction_fonction(enfant, annee, nomenclature)
  l_fonction.append(df_fonction)
  l_contfon.append(df_contfon)
  l_fonction_ref.append(df_fonction_ref)

 pd.concat(lnature_chapitre).to_parquet(f'./parquet/Nature_Chapitre_{annee}.parquet')
 pd.concat(lnature_compte).to_parquet(f'./parquet/Nature_Compte_{annee}.parquet')
 pd.concat(l_fonction).to_parquet(f'./parquet/Fonction_Chapitre_{annee}.parquet')
 pd.concat(l_contfon).to_parquet(f'./parquet/Fonction_Compte_{annee}.parquet')
 pd.concat(l_fonction_ref).to_parquet(f'./parquet/Fonction_Referentiel_{annee}.parquet')


In [13]:
xml_to_parquet_plan_de_compte('2020')

In [2]:
fch = pd.read_parquet('./parquet/Fonction_Chapitre_2020.parquet')
fco = pd.read_parquet('./parquet/Fonction_Compte_2020.parquet')
fcr = pd.read_parquet('./parquet/Fonction_Referentiel_2020.parquet')

nch = pd.read_parquet('./parquet/Nature_Chapitre_2020.parquet')
nco = pd.read_parquet('./parquet/Nature_Compte_2020.parquet')

In [35]:
print(fch.shape)
nco.head(1)

(177, 8)


,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,Exer,Nomenclature,PourEtatSeul
0,001,0,,,001,Solde d'exécution sect° d'investissement,Solde d'exécution de la section d'investisseme...,0,,,001,None,None,None,2020,M14-M14_COM_500_3500,None


In [47]:
fco['Supprime'].value_counts()

Supprime
1    4
Name: count, dtype: int64